In [103]:
import seaborn as sns
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix
import random
from scipy.stats import boxcox

In [12]:
housing_data = pd.DataFrame.from_csv('cleaned.csv')

In [86]:
# Calculate the baseline for linear regression
prices = housing_data.filter(['SalePrice'])
avg_price = prices.mean()

In [85]:
# Identify the relevant features

columns = housing_data.columns
column_dtypes = housing_data.dtypes
numerical_types = [np.int64, np.int32, np.float32, np.float64]

valid_col = [columns[i] for i in range(len(columns)) if column_dtypes[i] in numerical_types]
X = pd.DataFrame(housing_data, columns = valid_col)
corr = np.corrcoef(X, rowvar=False)

# Get SalePrice corrcoef with everything else
sale_price_corr_coef = corr[-1][:-1]

significant_col = []
significant_col_values = []
sig_threshold = 0.5 # arbitrarily setting it at >=0.5
print("Below is the correlation coefficients for SalePrice with each of the other numerical features:\n")
for i in range(len(sale_price_corr_coef)):
    corr_coef = sale_price_corr_coef[i]
    print("{}. {}: {}".format(i+1, valid_col[i], corr_coef))
    if abs(corr_coef) >= sig_threshold:
        significant_col.append(valid_col[i])
        significant_col_values.append(corr_coef)

print("\nBelow are the columns (and correlation coefficients) that have been deemed significant (absolute coefficient >= 0.5) and will be used " +
      "in the linear regression model")

for i in range(len(significant_col)):
    print("{}. {}: {}".format(i+1, significant_col[i], significant_col_values[i]))
    
feature_set = housing_data.filter(significant_col)



Below is the correlation coefficients for SalePrice with each of the other numerical features:

1. MSSubClass: -0.08353247161509614
2. LotFrontage: 0.20913730036170808
3. LotArea: 0.26395544844356883
4. OverallQual: 0.790870103285519
5. OverallCond: -0.07747076838683668
6. YearBuilt: 0.5229250946272648
7. YearRemodAdd: 0.5070873870879856
8. BsmtFinSF1: 0.38585196108705444
9. BsmtFinSF2: -0.013811048393577989
10. BsmtUnfSF: 0.2128527540442278
11. TotalBsmtSF: 0.6150306002600916
12. 1stFlrSF: 0.6056811685298111
13. 2ndFlrSF: 0.32045452502532074
14. LowQualFinSF: -0.025516478206112695
15. GrLivArea: 0.7088612518306577
16. BsmtFullBath: 0.22645875233836776
17. BsmtHalfBath: -0.016644134378481324
18. FullBath: 0.5606660611742321
19. HalfBath: 0.28532429636616874
20. BedroomAbvGr: 0.16817449421857777
21. KitchenAbvGr: -0.13580936177277583
22. TotRmsAbvGrd: 0.5338292886465184
23. Fireplaces: 0.4667621313668411
24. GarageCars: 0.6399663562678245
25. GarageArea: 0.622866688949611
26. WoodDeckSF

In [80]:
splits = [0.1, 0.2, 0.3, 0.4, 0.5]
random_states = [42, 1337, 420, 90210, 24]

In [109]:
def create_and_test_model(dataset, target, test_size, seed):
    X_train, X_test, y_train, y_test = train_test_split(dataset, target, test_size = test_size, random_state=seed)
    model = LinearRegression()
    model = model.fit(X=X_train, y=y_train)
    predictions = model.predict(X=X_test)
    MSE = sklearn.metrics.mean_squared_error(y_true = y_test, y_pred = predictions)
    RMSE = np.sqrt(MSE)
    num_targets = len(y_test)
    baseline_pred = [avg_price] * num_targets
    BaselineMSE = sklearn.metrics.mean_squared_error(y_true = y_test, y_pred = baseline_pred)
    BaselineRMSE = np.sqrt(BaselineMSE)
    return model, RMSE, BaselineRMSE

In [61]:
# Create and test model for linear regression for 5 different train_test splits

In [111]:
for i in range(5):
    test_size = splits[i]
    seed = random_states[i]
    model, RMSE, BaselineRMSE = create_and_test_model(feature_set, prices, test_size, seed)
    print("The RMSE for the split {} is {}".format(test_size, RMSE))
    improvement = (BaselineRMSE - RMSE) * 100 / BaselineRMSE 
    print("This model performs {:.2f}% better than the baseline model".format(improvement))
    print("\n")

The RMSE for the split 0.1 is 31899.344188135525
This model performs 59.70% better than the baseline model


The RMSE for the split 0.2 is 34524.61572284954
This model performs 55.12% better than the baseline model


The RMSE for the split 0.3 is 41071.36510190622
This model performs 45.40% better than the baseline model


The RMSE for the split 0.4 is 38010.6865852632
This model performs 54.15% better than the baseline model


The RMSE for the split 0.5 is 42517.34137133242
This model performs 47.46% better than the baseline model




In [63]:
# Calculate the baseline for KNN model



In [64]:
# Create and test KNN model for 5 different train_test splits

In [65]:
#add column 'After1970' to dataframe to specify if house was built after 1970 (True) or before (False)
housing_data['After1970'] = housing_data['YearBuilt'] > 1970

In [69]:
# Identify the relevant features
### NEED TO DO MORE FEATURE ENGINEERING HERE! ###
KNN_relevant_features = ['YearRemodAdd', 'OverallQual', 'GarageCars']

KNN_feature_set = housing_data.filter(KNN_relevant_features)

After1970 = housing_data.filter(['After1970'])

#determine strength of correlation for features of interest
# X = pd.DataFrame(housing_data,columns=['YearRemodAdd', 'OverallQual', 'GarageCars','YearBuilt']) 
# corr = np.corrcoef(X,rowvar=False) #rowvar = False b/c columns are features
# print (corr)

Correlations with YearBuilt are as follows: YearRemodAdd (.59), OverallQual (.57), GarageCars (.54)

In [79]:
def create_and_test_KNN_model(dataset, target, split, seed):
    np.random.seed(seed)
    index = np.random.rand(len(dataset)) < split[0]
    train_features = dataset[index]
    test_features = dataset[~index]
    
    train_target = target[index]
    test_target = target[~index]
    
    model = KNeighborsClassifier(n_neighbors=1)
    
    model = model.fit(X=train_features, y=train_target)
    predictions = model.predict(X=test_features)
    confusion = confusion_matrix(test_target, predictions)
    
    return model, confusion

In [83]:
#use same splits as in linear regression above
#ravel converts column to 1_D array (avoids a data type conversion warning)

for i in range(0, 5):
    split = splits[i] 
    seed = random_states[i]
    model, confusion = create_and_test_KNN_model(KNN_feature_set, np.ravel(After1970), split, seed)
    print("Confusion matrix for split {} is: \n {}".format(split, confusion))

Confusion matrix for split (0.9, 0.1) is: 
 [[63 11]
 [ 9 68]]
Confusion matrix for split (0.8, 0.2) is: 
 [[116  23]
 [ 21 140]]
Confusion matrix for split (0.7, 0.3) is: 
 [[153  33]
 [ 31 204]]
Confusion matrix for split (0.6, 0.4) is: 
 [[217  27]
 [ 49 258]]
Confusion matrix for split (0.5, 0.5) is: 
 [[291  43]
 [ 39 357]]
